in this notebook we:
* test and build a function which copies the data to the new disk rescaling it to ~512 x 384 pixels

Hence we want to do:
* for propor ~1.3 (so 1.25, 1.3, 1.33)
 * push y to 384 and x accordingly (so x is ~512)
* else (so 1.78, 1.72)
 * push y to 384 so x is 684 and 660

in rescaling we follow ppleskov from https://gitlab.com/ppleskov/snapshot-serengeti/blob/master/resize.ipynb

In [1]:
from pathlib import Path
import json
import logging

import numpy as np
import pandas as pd

import os

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from tqdm import tqdm_notebook as tqdm
from PIL import Image

from joblib import Parallel, delayed

In [2]:
PATH_TO_IMG = Path("/data/Gabon_trainingData")

PATH_TO_MAIN = Path("/home/jupyter/")
PATH_TO_TRAIN_DF = PATH_TO_MAIN / "inspect_data_split_validation"

In [3]:
df = pd.read_csv(PATH_TO_TRAIN_DF / "train_valid_df.csv")
df.head()

,uniqueName,species,main_folder,is_valid,sub_folder
0,0000005.jpg,Human,Compressed Camera Trap Images,False,T33
1,0000007.jpg,Human,Compressed Camera Trap Images,False,T33
2,0000009.jpg,Human,Compressed Camera Trap Images,False,T33
3,0000013.jpg,Human,Compressed Camera Trap Images,False,T33
4,0000015.jpg,Human,Compressed Camera Trap Images,False,T33


In [4]:
filename = "sizes.npy"
sizes_from_file = np.load(filename).tolist()
len(sizes_from_file)

347120

In [5]:
df_sizes = pd.DataFrame(sizes_from_file, columns=["x", "y", "uniqueName"])
df_sizes["dim"] = df_sizes["x"]+" x "+df_sizes["y"]
df_sizes.head()

,x,y,uniqueName,dim
0,1088,816,0000005.jpg,1088 x 816
1,1088,816,0000007.jpg,1088 x 816
2,1088,816,0000009.jpg,1088 x 816
3,1088,816,0000013.jpg,1088 x 816
4,1088,816,0000015.jpg,1088 x 816


In [8]:
df_desc_sizes = pd.merge(df, df_sizes, how="left", on="uniqueName")
df_desc_sizes

,uniqueName,species,main_folder,is_valid,sub_folder,x,y,dim
0,0000005.jpg,Human,Compressed Camera Trap Images,False,T33,1088,816,1088 x 816
1,0000007.jpg,Human,Compressed Camera Trap Images,False,T33,1088,816,1088 x 816
2,0000009.jpg,Human,Compressed Camera Trap Images,False,T33,1088,816,1088 x 816
3,0000013.jpg,Human,Compressed Camera Trap Images,False,T33,1088,816,1088 x 816
4,0000015.jpg,Human,Compressed Camera Trap Images,False,T33,1088,816,1088 x 816
...,...,...,...,...,...,...,...,...
347115,1529566.jpg,Duiker_Red,allData,False,NouabaleTEAM,1280,1024,1280 x 1024
347116,1529571.jpg,Duiker_Red,allData,False,NouabaleTEAM,1280,1024,1280 x 1024
347117,1529572.jpg,Duiker_Red,allData,False,NouabaleTEAM,1280,1024,1280 x 1024
347118,1529575.jpg,Duiker_Red,allData,False,NouabaleTEAM,1280,1024,1280 x 1024


In [9]:
counts_by_source_dim = df_desc_sizes.groupby(["main_folder", "dim", "is_valid"]).size().reset_index(name="counts").sort_values(by=['main_folder','counts'],ascending=False)
counts_by_source_dim

,main_folder,dim,is_valid,counts
7,allData,2048 x 1536,False,123954
5,allData,1280 x 1024,False,60393
6,allData,1600 x 1200,True,9268
8,allData,2048 x 1536,True,6306
10,allData,2576 x 1496,True,4750
12,allData,3264 x 1832,True,2905
9,allData,2560 x 1920,True,734
13,allData,3264 x 2448,True,324
11,allData,2576 x 1984,True,41
3,Compressed Camera Trap Images,1088 x 816,False,88990


In [10]:
counts_by_source_dim_all_data = df_desc_sizes[df_desc_sizes["main_folder"] == "allData"]\
                                    .groupby(["dim", "sub_folder", "is_valid"]).size()\
                                    .reset_index(name="counts")\
                                    .sort_values(by=['sub_folder','counts'],ascending=False)
counts_by_source_dim_all_data

,dim,sub_folder,is_valid,counts
5,2576 x 1496,StephBrittainZSL,True,4750
7,3264 x 1832,StephBrittainZSL,True,2905
9,3264 x 2448,StephBrittainZSL,True,74
6,2576 x 1984,StephBrittainZSL,True,41
0,1280 x 1024,NouabaleTEAM,False,60393
1,1600 x 1200,LailaBahaaelDinPanthera,True,9268
3,2048 x 1536,LailaBahaaelDinPanthera,True,6306
4,2560 x 1920,LailaBahaaelDinPanthera,True,734
8,3264 x 2448,LailaBahaaelDinPanthera,True,250
2,2048 x 1536,KorupTEAM,False,123954


### Proportions

In [11]:
df_sizes["propor"] = round(df_sizes.x.apply(float) / df_sizes.y.apply(float),2)
df_sizes.propor.value_counts()

1.33    243005
1.25     60393
1.78     38931
1.72      4750
1.30        41
Name: propor, dtype: int64

In [12]:
df_sizes.groupby(["propor", "dim"]).size()

propor  dim        
1.25    1280 x 1024     60393
1.30    2576 x 1984        41
1.33    1088 x 816     102418
        1600 x 1200      9268
        1920 x 1440         1
        2048 x 1536    130260
        2560 x 1920       734
        3264 x 2448       324
1.72    2576 x 1496      4750
1.78    3264 x 1832      2905
        3840 x 2160     36026
dtype: int64

In [37]:
df_sizes

,x,y,uniqueName,dim,propor
0,1088,816,0000005.jpg,1088 x 816,1.33
1,1088,816,0000007.jpg,1088 x 816,1.33
2,1088,816,0000009.jpg,1088 x 816,1.33
3,1088,816,0000013.jpg,1088 x 816,1.33
4,1088,816,0000015.jpg,1088 x 816,1.33
...,...,...,...,...,...
347115,1280,1024,1529566.jpg,1280 x 1024,1.25
347116,1280,1024,1529571.jpg,1280 x 1024,1.25
347117,1280,1024,1529572.jpg,1280 x 1024,1.25
347118,1280,1024,1529575.jpg,1280 x 1024,1.25


### Code for rescaling

In [26]:
import os
import piexif

from tqdm.notebook import tqdm
from PIL import Image

from joblib import Parallel, delayed

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [42]:
PATH_TO_IMG_RESC = Path("/data_rescaled/")

In [21]:
# ! pip install piexif

In [34]:
images_x_y = df_desc_sizes[["uniqueName", "x", "y"]].values.tolist()
images_x_y[:4]

[['0000005.jpg', '1088', '816'],
 ['0000007.jpg', '1088', '816'],
 ['0000009.jpg', '1088', '816'],
 ['0000013.jpg', '1088', '816']]

In [37]:
for image_x_y in tqdm(images_x_y[:4]):
    print(image_x_y[0])

0000005.jpg
0000007.jpg
0000009.jpg
0000013.jpg



In [ ]:
exceptions = []
for image_x_y in tqdm(images_x_y):
    try:
        filename = image_x_y[0]
        new_filename = "resc_" + filename
        if os.path.isfile(PATH_TO_IMG_RESC / new_filename):
            continue
        img = Image.open(PATH_TO_IMG / filename)
        y = 384
        scale = 384 / int(image_x_y[2])
        x = round(scale * int(image_x_y[1]))
        img = img.resize((x, y), Image.ANTIALIAS)
        img.save(PATH_TO_IMG_RESC / new_filename)
        piexif.transplant(PATH_TO_IMG / filename, PATH_TO_IMG_RESC / new_filename)
    except Exception as e:
        exceptions.append([image_x_y, e])

In [58]:
ls /data_rescaled

lost+found/       resc_0000009.jpg  resc_0000017.jpg  resc_1529575.jpg
resc_0000005.jpg  resc_0000013.jpg  resc_0000021.jpg  resc_1529576.jpg
resc_0000007.jpg  resc_0000015.jpg  resc_1529572.jpg


In [52]:
for image_x_y in tqdm(images_x_y[-4:]):
    print(image_x_y)

['1529571.jpg', '1280', '1024']
['1529572.jpg', '1280', '1024']
['1529575.jpg', '1280', '1024']
['1529576.jpg', '1280', '1024']



In [61]:
[[Image.open(PATH_TO_IMG_RESC / file).size, file] for file in os.listdir(PATH_TO_IMG_RESC) if "." in file]

[[(512, 384), 'resc_0000017.jpg'],
 [(512, 384), 'resc_0000009.jpg'],
 [(512, 384), 'resc_0000013.jpg'],
 [(512, 384), 'resc_0000015.jpg'],
 [(512, 384), 'resc_0000007.jpg'],
 [(480, 384), 'resc_1529575.jpg'],
 [(512, 384), 'resc_0000021.jpg'],
 [(480, 384), 'resc_1529576.jpg'],
 [(480, 384), 'resc_1529572.jpg'],
 [(512, 384), 'resc_0000005.jpg']]

In [6]:
sizes_from_file[:4]

[['1088', '816', '0000005.jpg'],
 ['1088', '816', '0000007.jpg'],
 ['1088', '816', '0000009.jpg'],
 ['1088', '816', '0000013.jpg']]